In [17]:
import pandas as pd

In [18]:
hist_data = pd.read_csv("hist_data.csv")
class_hist_data = pd.read_csv('user_class_df_all_data.csv')

In [19]:
train = train.merge(class_train, how='left', on='buyer_id')
hist_data = hist_data.merge(class_hist_data, how='left', on='buyer_id')

In [20]:
train

,buyer_id,pav_order_id,created,item_id,count,price_sold,flag_weight_goods,weight,Unnamed: 0_x,group_id_x,Unnamed: 0_y,group_id_y
0,95203091,98506637863,2021-07-01 00:03:44,202808329,1.000,79.990,False,11.14,17020,4,17020,4
1,95203091,98506637863,2021-07-01 00:03:44,202953905,1.072,44.945,True,11.14,17020,4,17020,4
2,95203091,98506637863,2021-07-01 00:03:44,203566452,1.000,69.990,False,11.14,17020,4,17020,4
3,95203091,98506637863,2021-07-01 00:03:44,202820143,1.972,41.295,True,11.14,17020,4,17020,4
4,95203091,98506637863,2021-07-01 00:03:44,204400422,1.000,269.990,False,11.14,17020,4,17020,4
...,...,...,...,...,...,...,...,...,...,...,...,...
3276304,95619205,4620221287,2021-09-07 17:54:27,202807021,2.000,69.890,False,16.52,483,3,483,3
3276305,95619205,4620221287,2021-09-07 17:54:27,203580843,2.000,36.890,False,16.52,483,3,483,3
3276306,95619205,4620221287,2021-09-07 17:54:27,210976455,1.000,89.990,False,16.52,483,3,483,3
3276307,95619205,4620221287,2021-09-07 17:54:27,202931865,1.000,89.990,False,16.52,483,3,483,3


In [21]:
class_hist_data_dict = {i[0]: i[1] for i in class_hist_data[['buyer_id', 'group_id']].values}

In [ ]:
import json
from os.path import join
import pandas as pd
import numpy as np
import gc
from collections import Counter 
from tqdm import tqdm 


def make_coocurs_dict(data):
    tmp = (
        data[['item_id', 'pav_order_id']]
        .sort_values(['item_id', 'pav_order_id'])
        .merge(data[['item_id', 'pav_order_id']], how='left', on=['pav_order_id'], suffixes=('', '_left'))
    )
    tmp = tmp[tmp['item_id'] != tmp['item_id_left']].copy()
    tmp1 = tmp.groupby(['item_id'])['item_id_left'].agg(lambda x: Counter(x).most_common(20))
    tmp2 = data.groupby('item_id')['pav_order_id'].count().reset_index()
    base = dict(zip(tmp2.item_id, tmp2.pav_order_id))
    most_freq_dict = {k: [(x[0], (x[1]+0.3)/(50+base[k])) for x in v] for (k, v) in tmp1.iteritems()}
    del tmp1, tmp
    gc.collect()
    return most_freq_dict

def freq_by_person(data, class_dict):
    js = {}
    for group_id in tqdm(data['group_id'].unique()):
        
        data1 = data.loc[data.group_id == group_id]
        js[int(group_id)] = make_coocurs_dict(data1)
        #print('next buyer {}'.format(buyer))
    return js
    

personal_test = freq_by_person(hist_data, class_hist_data_dict)
with open('class_rec_hist.json', 'w', encoding='utf8') as json_file:
    json.dump(personal_test, json_file, ensure_ascii=False, indent=4)

100%|██████████| 7/7 [00:40<00:00,  5.77s/it]
